In [11]:
import pandas as pd
import sqlalchemy as sa
from abc import ABCMeta, abstractmethod
from rpy2.robjects import r, pandas2ri, globalenv

In [2]:
def create_connection(server, database, username, password, port):
    conn = f'postgresql+psycopg2://{username}:{password}@{server}:{port}/{database}'
    return sa.create_engine(conn)

In [3]:
class Stg_Model(metaclass=ABCMeta):
    @classmethod
    def extract(cls, year_start, year_end, uf, info_sys):
        r(f"""
            rm(list = ls(all = TRUE))
            remotes::install_github("rfsaldanha/microdatasus")

            library(microdatasus)

            dados <- fetch_datasus(
                year_start = {year_start}, 
                year_end = {year_end}, 
                uf = "{uf}", 
                information_system = "{info_sys}")
        """)

    @abstractmethod
    def treat(self):
        raise NotImplementedError

    @classmethod
    def load(cls, con, schema, table_name):
        pandas2ri.activate()
    
        rdf = globalenv['dados']

        pd_df = pandas2ri.rpy2py(rdf)
        
        pd_df.to_sql(con=con, schema=schema, name=table_name, if_exists='replace', index=False)

    @classmethod
    def run(cls, year_start, year_end, uf, info_sys, conn_params):
        cls.extract(year_start, year_end, uf.upper(), info_sys)

        cls.treat()

        con = create_connection(
            server='localhost', 
            database='saude_mental', 
            username='postgres', 
            password='postgres', 
            port=5432)

        cls.load(con=con, schema='stg', table_name='sim_2010_2020')


class Stg_SIM(Stg_Model):
    @classmethod
    def treat(cls):
        r("dados <- process_sim(dados)")

In [ ]:
Stg_SIM.extract(
    year_start=2010,
    year_end=2020,
    uf='ES',
    info_sys='SIM-DO'
)

In [5]:
Stg_SIM.treat()

In [6]:
con = create_connection(
            server='localhost', 
            database='saude_mental', 
            username='postgres', 
            password='postgres', 
            port=5432)

In [46]:
rdf = globalenv['dados']

pd_df = pandas2ri.rpy2py(rdf).astype(str)

type(pd_df)

pandas.core.frame.DataFrame

In [47]:
for col in pd_df.columns:
    pd_df.loc[pd_df[col] == 'NA_character_', col] = None

In [48]:
pd_df

,CONTADOR,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,DTNASC,SEXO,RACACOR,ESTCIV,...,IDADEmeses,IDADEanos,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
1,1,1,Não Fetal,2010-01-08,1340,None,1923-12-04,Feminino,Parda,Casado,...,None,86,ATIVO,MUNIC,Cachoeiro de Itapemirim,Espírito Santo,-20.85011,-41.11192,36,878.179
2,2,1,Não Fetal,2010-05-03,0830,None,1942-02-06,Masculino,Preta,Casado,...,None,68,ATIVO,MUNIC,Afonso Cláudio,Espírito Santo,-20.07704,-41.12546,361,941.188
3,3,1,Não Fetal,2010-05-07,0025,None,1932-02-11,Feminino,Branca,Viúvo,...,None,78,ATIVO,MUNIC,Brejetuba,Espírito Santo,-20.15703,-41.28968,750,354.404
4,4,1,Não Fetal,2010-06-25,0400,None,1950-08-07,Feminino,Branca,Casado,...,None,59,ATIVO,MUNIC,Brejetuba,Espírito Santo,-20.15703,-41.28968,750,354.404
5,5,1,Não Fetal,2010-07-11,1800,None,1928-07-11,Feminino,Branca,Viúvo,...,None,82,ATIVO,MUNIC,Brejetuba,Espírito Santo,-20.15703,-41.28968,750,354.404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254255,1555135,1,Não Fetal,2020-12-31,1200,None,1956-09-10,Masculino,Branca,Solteiro,...,None,64,ATIVO,MUNIC,Cariacica,Espírito Santo,-20.29468,-40.39056,37,279.859
254256,1555138,1,Não Fetal,2020-12-31,1630,None,1940-04-06,Feminino,Branca,None,...,None,80,ATIVO,MUNIC,Marilândia,Espírito Santo,-19.41281,-40.55095,146,309.018
254257,1555152,1,Não Fetal,2020-12-31,0620,None,1959-02-11,Masculino,Parda,Casado,...,None,61,ATIVO,MUNIC,Vila Velha,Espírito Santo,-20.33837,-40.29396,3,209.965
254258,1555739,2,Não Fetal,2020-12-11,1710,None,1944-03-18,Masculino,Parda,Casado,...,None,76,ATIVO,MUNIC,Cariacica,Espírito Santo,-20.29468,-40.39056,37,279.859


In [49]:
pd_df.to_sql(con=con, schema='stg', name='sim_2010_2020', if_exists='replace', index=False)

277

In [ ]:
Stg_SIM.load(con=con, schema='stg', table_name='sim_2010_2020')